## Test Run of Proposed Protocol

Given the security deficiencies of using Python's `random` module, here is an alternative way for $N$ users to generate a shared random $D$ value for each run of the SAFE Protocol.

Steps:
* each user uses the `secret` module (which uses a Crytographically-Secure PRNG) to generate a random integer.
* this integer is mapped to a float (in the case of `0` or `nan`, some default float value is used as their secret share.)
* users do additive secret sharing of their float values to come up with a shared $D$ value.

### Example Test Run 

Let $N = 10$. First we **define helper functions**. 

In [5]:
# Helper Functions To Convert Integers To Floats

"""
* Key Resources: (code adapted from)
# https://stackoverflow.com/questions/8751653/how-to-convert-a-binary-string-into-a-float-value
# https://stackoverflow.com/questions/25099626/convert-scientific-notation-to-float
# https://stackoverflow.com/questions/30971079/how-to-convert-an-integer-to-a-list-of-bits
"""

import math
from codecs import decode
import struct

DEFAULT_FLOAT = 1569832.524 # NOTE- just choosing some value. Not sure if there's a better way to do it.

def int_to_bin(num: int) -> str:
    """ Convert integer to binary string representation. """
    list_of_bits = [1 if num & (1 << (31-n)) else 0 for n in range(32)]
    string_version = ''.join([str(bit) for bit in list_of_bits])
    return string_version

def bin_to_float(byte_string: str) -> str:
    """ Convert binary string to a float. """
    float_bytes = int(byte_string, 2).to_bytes(4, 'big')  
    # see https://docs.python.org/3/library/struct.html for manipulation of params
    float_string =  format(struct.unpack('>f', float_bytes)[0], 'f')
    return float(float_string)

def int_to_float(num: int) -> float:
    bits = int_to_bin(num)
    result = abs(bin_to_float(bits))
    if result == 0.0 or math.isnan(result):
        return DEFAULT_FLOAT
    else:
        return result 

# Test examples
num = 4140206672 # doesn't return nan
print(f'For the output: {num}, we have the float: {int_to_float(num)}')

zero_num = 2195631465 # `result` in `int_to_float` will be 0.0
print(f'For the output: {zero_num}, we have the float: {int_to_float(zero_num)}')

For the output: 4140206672, we have the float: 2.0134376733753925e+33
For the output: 2195631465, we have the float: 1569832.524


Next, we have 10 users generate cryptographically-secure psuedo-random integers.

In [ ]:
import secrets